https://matplotlib.org/mpl-probscale/tutorial/getting_started.html
https://matplotlib.org/mpl-probscale/tutorial/closer_look_at_viz.html



In [1]:
%matplotlib inline

import numpy
import pandas
import seaborn
import probscale
import warnings

warnings.simplefilter('ignore')

DAY = pandas.offsets.Day()
WEEK = DAY * 7
MONTH = WEEK * 4

seaborn.set_style('ticks')

In [6]:
df = (
    pandas.read_csv('rainfall.csv', parse_dates=['date'])
        .assign(precip_mm=lambda df: numpy.where(df['hpcp'] > 100, numpy.nan, df['hpcp']))
        .set_index(['station', 'date'])['precip_mm']
)
df.head()

station      date               
COOP:366889  1948-01-01 01:00:00    0.00
             1948-01-01 11:00:00    0.25
             1948-01-01 12:00:00    0.51
             1948-01-01 13:00:00    3.05
             1948-01-01 14:00:00    2.79
Name: precip_mm, dtype: float64

In [9]:
rolling_totals = (
    df.xs('COOP:366889', level='station')
        .to_frame()
        .assign(daily=lambda df: df['precip_mm'].rolling(DAY).sum())
        .assign(weekly=lambda df: df['precip_mm'].rolling(WEEK).sum())
        .assign(monthly=lambda df: df['precip_mm'].rolling(MONTH).sum())
        .assign(semi_annual=lambda df: df['precip_mm'].rolling(MONTH * 6).sum())
        .assign(wy=lambda df: numpy.where(df.index.month >= 10, df.index.year + 1, df.index.year))
)

totals = (
    rolling_totals
        .drop(['wy'], axis='columns')
        .resample('1A')
        .max()
        .assign(year=lambda df: df.index.year)
        .reset_index(drop=True)
        .set_index('year')
        .rename(columns=lambda c: (c, 'yr'))
        .join(
            rolling_totals
                .groupby(['wy'])
                .max()
                .rename(columns=lambda c: (c, 'wy')),
            lsuffix='_yr', rsuffix='_wy'
        )
        .sort_index(axis='columns')
        .rename_axis(('window', 'yeartype'), axis='columns')
        .drop(['precip_mm'], axis='columns')
        .stack(level='window')
        .stack(level='yeartype')
        .rename('max_precip_mm')
        .to_frame()
        .reset_index()
)
totals.head(15)

ValueError: window must be an integer

In [8]:
windows = ['daily', 'weekly', 'monthly', 'semi_annual']
fg = (
    seaborn.FacetGrid(totals, col='window', col_wrap=2, col_order=windows,
                      hue='yeartype', hue_order=['yr', 'wy'],
                      sharey=False, aspect=2)
        .map(probscale.probplot, 'max_precip_mm')
        .set_axis_labels(x_var='Non-Exceedance Probability (%)', y_var='Precip (mm)')
        .add_legend()
)



NameError: name 'totals' is not defined